In [1]:
import numpy as np

In [2]:
class Graph:
    def __init__(self, directed = False):
        self.directed = directed
        self.graph_dict = {}
        
    def get_verticies(self):
        return list(self.graph_dict.keys())
        
    def add_vertex(self, new_vertex):
        self.graph_dict[new_vertex.value] = new_vertex
        
    def add_edge(self, from_vertex, to_vertex, weight = 0):
        from_vertex.add_edge(to_vertex, weight)
        if not self.directed:
            to_vertex.add_edge(from_vertex, weight)

class Vertex:
    def __init__(self, value):
        self.value = value      
        self.edges = {}
        
    def get_value(self):
        return self.value    
        
    def add_edge(self, to_vertex, weight):
        self.edges[to_vertex] = weight
        
    def get_edges(self):
        return self.edges
    
    

        

In [3]:
ex1 = Graph()
a = Vertex('A')
b = Vertex('B')
c = Vertex('C')
d = Vertex('D')
e = Vertex('E')
f = Vertex('F')
g = Vertex('G')
h = Vertex('H')

ex1.add_vertex(a)
ex1.add_vertex(b)
ex1.add_vertex(c)
ex1.add_vertex(d)
ex1.add_vertex(e)
ex1.add_vertex(f)
ex1.add_vertex(h)
ex1.add_vertex(g)


In [4]:
class Vertex2(Vertex):
    def __init__(self, value):
        super().__init__(value)
        self.easy_edges = {}
        
    def add_edge(self, to_vertex, weight):
        super().add_edge(to_vertex, weight)
        self.easy_edges[self.value + to_vertex.value] = weight
        
    def get_easy_edges(self):
        return self.easy_edges

class Graph2(Graph):
    def __init__(self, directed = False):
        super().__init__(directed)
        self.edges = {}
        
    def add_edge(self, from_vertex, to_vertex, weight = 0):
        super().add_edge(from_vertex, to_vertex, weight)
        self.edges[from_vertex.value + to_vertex.value] = weight
    
    def get_edges(self):
        return self.edges
    
    def kruskal(self):
        points = self.get_verticies()
        edges = sorted(self.get_edges(), key=lambda x:self.edges[x])
        used_edges = []
        while len(points) > 1:
            edge = edges.pop(0)
            i1, i2 = points.index(list(filter(lambda x: edge[0] in x, points))[0]), points.index(list(filter(lambda x: edge[1] in x, points))[0])
            if i1 == i2:
                continue
            points[i1] += points[i2]
            points.pop(i2)
            used_edges.append(edge)
        total_weight = sum([self.edges[edge] for edge in used_edges]) 
        return used_edges, total_weight
    
    def get_other_vertex(self, current_vertex, edge):
        return list(filter(lambda x: x not in current_vertex, edge))[0]
    
    def prim(self, start_vertex):
        points = [start_vertex.value]
        used_edges = []
        edges = list(start_vertex.easy_edges.items())
        total_weight = 0
        while len(points) < len(self.get_verticies()):
            edges = sorted(edges, key = lambda x: x[1])
            weight = edges[0][1]
            edge = edges.pop(0)[0]
            if edge[0] in points and edge[1] in points:
                continue
            to_vertex = self.get_other_vertex(points, edge)
            edges += list(self.graph_dict[to_vertex].easy_edges.items())
            points.append(to_vertex)
            used_edges.append(edge)
            total_weight += weight
        return used_edges, total_weight
    
    def djikstra(self, start_vertex, end_vertex):
        djik_dict = {x:[0, 0] for x in self.get_verticies()}
        count = 1
        djik_dict[start_vertex.value][0] = count
        current_vertex = start_vertex.value
        while not djik_dict[end_vertex.value][0]:
            edges = list(self.graph_dict[current_vertex].easy_edges.items())
            for edge in edges:
                to_vertex = self.get_other_vertex(current_vertex, edge[0])
                if (not djik_dict[to_vertex][0] and not djik_dict[to_vertex][1]) or djik_dict[to_vertex][1] > djik_dict[current_vertex][1] + edge[1]:
                    djik_dict[to_vertex][1] = djik_dict[current_vertex][1] + edge[1]
            count += 1
            current_vertex = sorted(djik_dict, key=lambda x: (djik_dict[x][0], djik_dict[x][1] == 0, djik_dict[x][1]))[0]
            djik_dict[current_vertex][0] = count
        return djik_dict, edges, current_vertex




- Graph1 just basic
- Graph2 add djikstra, prim and kruskal
- Graph3 allows graph to be passed in, adds functionality of print graph, finds fastest route to return xy coordinates

In [5]:
class Vertex3(Vertex2):
    def __init__(self, value, coordinates):
        super().__init__(value)
        self.coordinates = coordinates
        self.x = coordinates[0]
        self.y = coordinates[1]
        self.strco = str(self.x) + ',' + str(self.y)
        self.g = None
        self.h = None
        self.f = None
        self.parent = None
        self.visited = False
        
    def add_edge(self, to_vertex, weight):
        Vertex.add_edge(self, to_vertex, weight)
        self.easy_edges[f'{self.x},{self.y}/{to_vertex.x},{to_vertex.y}'] = weight
        
    def add_gcost(self, current_vertex, previous_vertex):
        self.g = 0 if previous_vertex == None else previous_vertex.edges[current_vertex] + previous_vertex.f

        
    def add_hcost(self, from_vertex, end_vertex):
        self.h = self.get_distance(from_vertex, end_vertex)
        
    def add_fcost(self, start_node = False):
        if (self.g and self.h) or start_node:
            self.f = self.g + self.h
            
    def add_parent(self, parent_vertex):
        self.parent = parent_vertex
            
    def get_fcost(self):
        return self.f
    
    def get_distance(self, f, t):   
        return round((abs(f.x - t.x) ** 2 + abs(f.y - t.y) ** 2) ** 0.5, 2)
    
    def add_costs(self, start, end, current, previous = None):
        self.add_gcost(current, previous)
        self.add_hcost(current, end)  
        if start == current:
            self.add_fcost(start_node = True)
        elif current == end:
            self.add_fcost(start_node = True)
            self.add_parent(previous)
        else:
            self.add_fcost()
            self.add_parent(previous)
        
class Graph3(Graph2):
    def __init__(self, directed = False):
        super().__init__(directed)
        self.graph = []
        self.start = None
        self.end = None
        
    def add_edge(self, from_vertex, to_vertex, weight = 0):
        Graph.add_edge(self, from_vertex, to_vertex, weight)
        self.edges[f'{from_vertex.x},{from_vertex.y}/{to_vertex.x},{to_vertex.y}'] = weight
        
    def toco(self, string):
        return [int(n) for n in string.split(',')]
    
    def tocos(self, string):
        return [self.toco(x) for x in string.split('/')]
    
    def tost(self, lst):
        return str(lst[0]) + ',' + str(lst[1]) 
    
    def get_other_vertex(self, current_vertex, edge):
        return self.tost(list(filter(lambda x: x not in [current_vertex.coordinates], self.tocos(edge)))[0])
    
    def add_edge_from_graph(self, i1, i2, shape):
        from_vertex = self.graph_dict[self.tost([i1, i2])]
        if i1 > 0:
            self.add_edge(from_vertex, self.graph_dict[self.tost([i1 - 1, i2])], 1)
        if i1 < shape[0] - 1:
            self.add_edge(from_vertex, self.graph_dict[self.tost([i1 + 1, i2])], 1)
        if i2 > 0:
            self.add_edge(from_vertex, self.graph_dict[self.tost([i1, i2 - 1])], 1)
        if i2 < shape[1] - 1:
            self.add_edge(from_vertex, self.graph_dict[self.tost([i1, i2 + 1])], 1)
        
    def make_graph(self, graph_string):
        self.graph = np.array([[x for x in y] for y in graph_string.split('\n')[1:-1]])
        shape = self.graph.shape
        for i1 in range(shape[0]):
            for i2 in range(shape[1]):
                value = int(self.graph[i1,i2]) if self.graph[i1,i2].isdigit() else self.graph[i1,i2]
                self.add_vertex(value, str(i1) + ',' + str(i2))
                if value == 'S':
                    self.start = str(i1) + ',' + str(i2)
                elif value == 'E':
                    self.end = str(i1) + ',' + str(i2) 
        for i1 in range(shape[0]):
            for i2 in range(shape[1]):
                self.add_edge_from_graph(i1, i2, shape)

    def astar(self):
        al = []
        start_vertex, end_vertex = self.graph_dict[self.start], self.graph_dict[self.end]
        to_visit = [self.graph_dict[self.start]]
        start_vertex.add_costs(start_vertex, end_vertex, start_vertex)
        while to_visit:
            to_visit = sorted(to_visit, key=lambda x: (x.f, x.h))
            current_node = to_visit.pop(0)
            current_node.visited = True
            if current_node.value == 'E':
                break
            new_nodes = current_node.get_edges().keys()
            for node in new_nodes:
                al.append(node)
                if not node.visited:
                    if node.value == 1:
                        node.visited = True
                    elif node.get_fcost():
                        if node.f > current_node.f + current_node.edges[node]:
                            node.add_costs(start_vertex, end_vertex, node, previous = current_node)
                            to_visit.append(node)
                    else:
                        node.add_costs(start_vertex, end_vertex, node, previous = current_node)
                        to_visit.append(node)
        
        current_node = self.graph_dict[self.end]
        parents = [current_node]
        while current_node.value != 'S':
            parents.append(current_node.parent)
            current_node = current_node.parent
        return [p.strco for p in parents][::-1]
    
    def add_vertex(self, value, coordinates):
        self.graph_dict[coordinates] = Vertex3(value, [int(n) for n in coordinates.split(',')])

# astar - gcost generated as go - from last vertex, then comapre with already there, if better imporove, set arent accordingly
# if all f cost same, go to hcost

In [6]:
# g4 = Graph3()
# g4.make_graph(o)
# g4.astar()


g = '''
00000000000
00001E00000
00001111100
00000000000
00000000S00
00000000000
'''
g5 = Graph3()
g5.make_graph(g)
g5.astar()



['4,8', '4,9', '3,9', '2,9', '1,9', '1,8', '1,7', '1,6', '1,5']

In [7]:
o = """
S0110
01000
01010
00010
0001E
"""
def to_arr(s):
    return np.array([[x for x in y] for y in s.split('\n')[1:-1]])
u = to_arr(o)
p = u.shape

In [8]:
ex2 = Graph2()

a = Vertex2('A')
b = Vertex2('B')
c = Vertex2('C')
d = Vertex2('D')
e = Vertex2('E')
f = Vertex2('F')
g = Vertex2('G')
h = Vertex2('H')

ex2.add_vertex(a)
ex2.add_vertex(b)
ex2.add_vertex(c)
ex2.add_vertex(d)
ex2.add_vertex(e)
ex2.add_vertex(f)
ex2.add_vertex(h)
ex2.add_vertex(g)

ex2.add_edge(c, a, 8)
ex2.add_edge(a, d, 18)
ex2.add_edge(d, c, 8)
ex2.add_edge(b, a, 8)
ex2.add_edge(c, f, 12)
ex2.add_edge(e, b, 15)
ex2.add_edge(d, e, 6)
ex2.add_edge(f, d, 6)
ex2.add_edge(b, d, 7)
ex2.add_edge(e, h, 8)
ex2.add_edge(g, d, 9)
ex2.add_edge(f, d, 6)
ex2.add_edge(h, f, 9)
ex2.add_edge(h, g, 8)

In [9]:
o = """
S0110
01000
01010
00010
0001E
"""
def to_arr(s):
    return np.array([[x for x in y] for y in s.split('\n')[1:-1]])
u = to_arr(o)
p = u.shape
u[1,2]

'0'

In [10]:
# d = {}
# for i1 in range(3):
#     for i2 in range(4):
#         d[[i1, i2]] = Vertex([i1, i2])

In [11]:
def make_graph(width, height, start = None, end = None, obstacles = None):
    graph = np.array(['0'] * (width * height)).reshape(height, width)
    if start:
        graph[start[0], start[1]] = 'S'
    if end:
        graph[end[0], end[1]] = 'E'
    if obstacles:
        for ob in obstacles:
            graph[ob[0], ob[1]] = '1'
    return graph

def make_string(arr):
    lines = [''.join(line) + '\n' for line in arr]
    string = ''.join(lines)
    return '\n' + string + '\n'
     